In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

We will use the story of alice and will generate a sentence similar to Alice

In [3]:
class Dictionary(object):
    def __init__(self):
        # We will have two dictionaries
        self.wordToIndex = {}
        self.indexToWord = {}
        self.index = 0
    
    def add_word(self, word):
        if word not in self.wordToIndex:
            self.wordToIndex[word] = self.index
            self.indexToWord[self.index] = word
            self.index += 1
            
    def __len__(self):
        return len(self.wordToIndex)

In [8]:
class TextProcess(object):
    def __init__(self):
        self.dictionary = Dictionary()
        
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
                    
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.wordToIndex[word]
                    index += 1
        # Find out how many batches we need
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches * batch_size]
        # return {batch_size, num_batches}
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [6]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [9]:
corpus = TextProcess()

In [10]:
rep_tensor = corpus.get_data("./src/alice.txt", batch_size)

In [12]:
rep_tensor.shape

torch.Size([20, 1484])

In [13]:
vocab = corpus.dictionary

In [14]:
len(vocab)

5290

In [15]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [16]:
class Testgenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Perform Word Embeding
        x = self.embed(x)
        
        # Reshape the input tensor
        #   x = x.view(batch_size, timesteps, embed_size)
        out, (h, c) = self.lstm(x, h)
        
        # Reshape the ouput from (samples, timesteps, output_features) to a shape appropriate for the fully connected layer
        #   (batch_size * timesteps , hidden_size)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h,c)

In [ ]:
model = TextGenerator(vocab_size, embed_size, hidden_size, )